<a href="https://colab.research.google.com/github/quirogaez/capstone/blob/main/notebooks/01_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/quirogaez/capstone/blob/main/notebooks/01_EDA.ipynb)

# 01_EDA – Análisis Exploratorio de Datos

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob

#Holi